In [1]:
# -----------------------------------------------------------
# 🧩 Tiling and Mask Generation from Orthomosaic + Shapefile
#
# This step processes each map set by:
# - Loading orthomosaic TIFF and corresponding shapefile
# - Cutting the raster into 1024×1024 tiles with 50% overlap (stride = 512)
# - Creating matching grayscale mask tiles from vector polygons
#     - Uses class_id directly from shapefile (0 = Water, 1 = Road, 2 = PVeg)
# - Saving:
#     - /tiled/images/       (chipped RGB images)
#     - /tiled/masks/        (grayscale mask tiles)
#     - tile_metadata.csv    (tile position tracking)
#     - raster_shape.txt     (original raster dimensions)
#
# Output per map:
# - /[MapName]/tiled/
#     ├── images/
#     ├── masks/
#     ├── tile_metadata.csv
#     └── raster_shape.txt
#
# Notes:
# - Tiles with no labeled features are skipped
# - Shapefile geometries are automatically projected to match the TIFF CRS
# - Mask pixels retain class_id values (no remapping)
# - Supports 3-class segmentation directly
# -----------------------------------------------------------

In [1]:
import os
import csv
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.windows import Window
from shapely.geometry import box
import geopandas as gpd
import cv2
from tqdm.notebook import tqdm

# --- List of all map sets (Updated) ---
all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek"
]

# --- Configuration ---
base_dir = "C:/QGIS"
chip_size = 1024
stride = 1024

# --- Main Loop Over All Maps ---
for map_folder in all_maps:
    print(f"🧩 Processing: {map_folder}")

    map_base = os.path.join(base_dir, map_folder)
    tif_path = os.path.join(map_base, f"{map_folder}.tiff")
    shp_path = os.path.join(map_base, f"{map_folder}.shp")

    output_img_dir = os.path.join(map_base, "tiled", "images")
    output_mask_dir = os.path.join(map_base, "tiled", "masks")
    metadata_path = os.path.join(map_base, "tiled", "tile_metadata.csv")
    shape_path = os.path.join(map_base, "tiled", "raster_shape.txt")

    # Create output folders
    os.makedirs(output_img_dir, exist_ok=True)
    os.makedirs(output_mask_dir, exist_ok=True)

    # Load raster and label data
    raster = rasterio.open(tif_path)
    labels = gpd.read_file(shp_path).to_crs(raster.crs)

    # Save raster shape
    with open(shape_path, "w") as f:
        f.write(f"{raster.height},{raster.width}")

    # Prepare CSV for tile metadata
    with open(metadata_path, mode="w", newline="") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["filename", "x", "y"])  # CSV header

        count = 0
        for y in tqdm(range(0, raster.height, stride), desc=f"Tiling {map_folder}"):
            for x in range(0, raster.width, stride):
                if x + chip_size > raster.width or y + chip_size > raster.height:
                    continue

                window = Window(x, y, chip_size, chip_size)
                transform = raster.window_transform(window)
                bounds = box(*rasterio.windows.bounds(window, raster.transform))
                intersecting = labels[labels.intersects(bounds)]

                if intersecting.empty:
                    continue

                # --- Save image tile ---
                image = raster.read(window=window)
                image = np.transpose(image[:3], (1, 2, 0))  # RGB
                img_filename = f"chip_{count}.png"
                img_path = os.path.join(output_img_dir, img_filename)
                cv2.imwrite(img_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

                # --- Save rasterized mask ---
                mask_shapes = [(geom, cls) for geom, cls in zip(intersecting.geometry, intersecting["class_id"])]
                mask = rasterize(
                    mask_shapes,
                    out_shape=(chip_size, chip_size),
                    transform=transform,
                    fill=0,
                    dtype=np.uint8
                )
                mask_path = os.path.join(output_mask_dir, img_filename)
                cv2.imwrite(mask_path, mask)

                writer.writerow([img_filename, x, y])
                count += 1

    print(f"✅ {count} tile+mask pairs created for {map_folder}\n")

🧩 Processing: Bear_Creek_20250112


Tiling Bear_Creek_20250112:   0%|          | 0/87 [00:00<?, ?it/s]

✅ 2047 tile+mask pairs created for Bear_Creek_20250112

🧩 Processing: Bear_Lane


Tiling Bear_Lane:   0%|          | 0/33 [00:00<?, ?it/s]

✅ 772 tile+mask pairs created for Bear_Lane

🧩 Processing: Flight_2


...

✅ 1810 tile+mask pairs created for SFLBC

🧩 Processing: Sugar_Refugia_20241112


Tiling Sugar_Refugia_20241112:   0%|          | 0/44 [00:00<?, ?it/s]

✅ 775 tile+mask pairs created for Sugar_Refugia_20241112

🧩 Processing: Wildcat_Creek


Tiling Wildcat_Creek:   0%|          | 0/63 [00:00<?, ?it/s]

✅ 559 tile+mask pairs created for Wildcat_Creek



In [8]:
# -----------------------------------------------------------
# 🔄 Convert Chipped Images and Masks to YOLOv8 Polygon Format
#
# This step processes each map set by:
# - Loading 1024×1024 image and mask tiles from the tiling step
# - Resizing both image and mask to 640×640 (YOLOv8 input size)
# - Converting mask regions to YOLO polygon labels (.txt)
#     - Each polygon retains its original class_id: 
#         - 0 = Water, 1 = Road, 2 = PVeg
# - Saving:
#     - /yolo_dataset_640/images/  (resized image tiles)
#     - /yolo_dataset_640/labels/  (polygon labels in YOLO format)
#
# Output per map:
# - /[MapName]/yolo_dataset_640/
#     ├── images/
#     └── labels/
#
# Notes:
# - This step does not perform a train/val split — that happens in the next step
# - Background-only tiles (no labeled polygons) are still included with empty label files
# - Class IDs are preserved directly from mask input; no remapping is needed
# -----------------------------------------------------------

In [2]:
import os
import cv2
import numpy as np
import random
from tqdm.notebook import tqdm

# --- Updated list of map sets ---
all_maps = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek"
]

# --- Configuration ---
base_dir = "C:/QGIS"
target_size = 640

# --- Helper function ---
def mask_to_polygons(mask):
    contours = {}
    for cls_id in np.unique(mask):
        binary = (mask == cls_id).astype(np.uint8)
        cnts, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours[cls_id] = cnts
    return contours

# --- Process each map set ---
for map_folder in all_maps:
    print(f"🧩 Processing: {map_folder}")

    base_map_dir = os.path.join(base_dir, map_folder)
    img_input_dir = os.path.join(base_map_dir, "tiled", "images")
    mask_input_dir = os.path.join(base_map_dir, "tiled", "masks")

    output_img_dir = os.path.join(base_map_dir, "yolo_dataset_640", "images")
    output_lbl_dir = os.path.join(base_map_dir, "yolo_dataset_640", "labels")

    img_train_dir = os.path.join(output_img_dir, "train")
    img_val_dir = os.path.join(output_img_dir, "val")
    lbl_train_dir = os.path.join(output_lbl_dir, "train")
    lbl_val_dir = os.path.join(output_lbl_dir, "val")

    # Create output directories
    for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
        os.makedirs(d, exist_ok=True)

    # --- List and shuffle tiles ---
    chip_files = sorted([f for f in os.listdir(img_input_dir) if f.endswith(".png")])
    random.shuffle(chip_files)

    # --- Split 70/30 ---
    split_idx = int(len(chip_files) * 0.7)
    train_files = chip_files[:split_idx]
    val_files = chip_files[split_idx:]

    background_only_count = 0

    # --- Processing ---
    for trainmode, file_list in [("train", train_files), ("val", val_files)]:
        img_out_dir = img_train_dir if trainmode == "train" else img_val_dir
        lbl_out_dir = lbl_train_dir if trainmode == "train" else lbl_val_dir

        for fname in tqdm(file_list, desc=f"Processing {trainmode} files ({map_folder})"):
            # --- Load and resize image ---
            img_path = os.path.join(img_input_dir, fname)
            img = cv2.imread(img_path)
            img_resized = cv2.resize(img, (target_size, target_size), interpolation=cv2.INTER_AREA)
            cv2.imwrite(os.path.join(img_out_dir, fname), img_resized)

            # --- Load and resize mask ---
            mask_path = os.path.join(mask_input_dir, fname)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask_resized = cv2.resize(mask, (target_size, target_size), interpolation=cv2.INTER_NEAREST)

            # --- Convert mask to YOLO polygon label ---
            contours = mask_to_polygons(mask_resized)
            label_path = os.path.join(lbl_out_dir, fname.replace(".png", ".txt"))

            label_written = False
            with open(label_path, "w") as f:
                for cls_id, cnts in contours.items():
                    for cnt in cnts:
                        if len(cnt) < 3:
                            continue

                        pts = cnt.reshape(-1, 2)
                        if len(pts) < 3:
                            continue

                        norm_pts = pts / target_size
                        coords = " ".join([f"{x:.6f} {y:.6f}" for x, y in norm_pts])
                        f.write(f"{cls_id} {coords}\n")
                        label_written = True

            if not label_written:
                background_only_count += 1

    print(f"✅ Completed {map_folder}: {len(train_files)} train / {len(val_files)} val tiles.")
    print(f"ℹ️ {background_only_count} background-only images in {map_folder}\n")

🧩 Processing: Bear_Creek_20250112


Processing train files (Bear_Creek_20250112):   0%|          | 0/1432 [00:00<?, ?it/s]

Processing val files (Bear_Creek_20250112):   0%|          | 0/615 [00:00<?, ?it/s]

✅ Completed Bear_Creek_20250112: 1432 train / 615 val tiles.
ℹ️ 0 background-only images in Bear_Creek_20250112

🧩 Processing: Bear_Lane


Processing train files (Bear_Lane):   0%|          | 0/540 [00:00<?, ?it/s]

...

Processing val files (Sugar_Refugia_20241112):   0%|          | 0/233 [00:00<?, ?it/s]

✅ Completed Sugar_Refugia_20241112: 542 train / 233 val tiles.
ℹ️ 0 background-only images in Sugar_Refugia_20241112

🧩 Processing: Wildcat_Creek


Processing train files (Wildcat_Creek):   0%|          | 0/391 [00:00<?, ?it/s]

Processing val files (Wildcat_Creek):   0%|          | 0/168 [00:00<?, ?it/s]

✅ Completed Wildcat_Creek: 391 train / 168 val tiles.
ℹ️ 0 background-only images in Wildcat_Creek



In [ ]:
# -----------------------------------------------------------
# 🗂️ Unified Dataset Creation for YOLOv8 Training (Multi-Set)
#
# This step constructs a single, merged dataset for training by:
# - Randomly selecting 6 of 7 available labeled tile sets
# - Merging all training and validation tiles into:
#     - images/train/
#     - images/val/
#     - labels/train/
#     - labels/val/
# - Renaming files to preserve set origin (e.g., Bear_Creek_chip_042.png)
# - Preserving background-only images with empty .txt labels
# - Creating manifest files for YOLOv8 training configuration
#
# Held-Out Test Set:
# - One dataset is held out (randomly) for evaluation and comparison
#
# Output:
# - /dataset/
#     ├── images/train/
#     ├── images/val/
#     ├── labels/train/
#     ├── labels/val/
#     └── manifests/
#         ├── train_files.txt
#         └── val_files.txt
#
# Notes:
# - Background-only tiles are retained (important for model balance)
# - Manifest files list relative paths for reproducibility and config use
# - Naming format ensures traceability of every image's origin set
# -----------------------------------------------------------

In [1]:
import os
import shutil
import random
from tqdm import tqdm

# --- Configuration ---
base_dir = "C:/QGIS"

all_sets = [
    "Bear_Creek_20250112",
    "Bear_Lane",
    "Flight_2",
    "Flight_2_25pct",
    "SFLBC",
    "Sugar_Refugia_20241112",
    "Wildcat_Creek"
]

# Output structure
dataset_dir = os.path.join(base_dir, "dataset")
img_train_dir = os.path.join(dataset_dir, "images", "train")
img_val_dir = os.path.join(dataset_dir, "images", "val")
lbl_train_dir = os.path.join(dataset_dir, "labels", "train")
lbl_val_dir = os.path.join(dataset_dir, "labels", "val")

for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    os.makedirs(d, exist_ok=True)

# --- Random split: 6 train sets + 1 holdout set ---
random.shuffle(all_sets)
train_sets = all_sets[:-1]
test_set = all_sets[-1]

print(f"✅ Training Sets ({len(train_sets)}): {train_sets}")
print(f"🧪 Held-Out Test Set: {test_set}")

# Manifest lists
train_list = []
val_list = []

# --- Merge training data ---
for set_name in tqdm(train_sets, desc="Merging training sets"):
    img_base = os.path.join(base_dir, set_name, "yolo_dataset_640", "images")
    lbl_base = os.path.join(base_dir, set_name, "yolo_dataset_640", "labels")

    for split in ["train", "val"]:
        img_in_dir = os.path.join(img_base, split)
        lbl_in_dir = os.path.join(lbl_base, split)

        img_out_dir = img_train_dir if split == "train" else img_val_dir
        lbl_out_dir = lbl_train_dir if split == "train" else lbl_val_dir

        if not os.path.exists(img_in_dir):
            print(f"⚠️ Missing: {img_in_dir} — skipping")
            continue

        for img_file in os.listdir(img_in_dir):
            if not img_file.endswith(".png"):
                continue

            base_name = img_file.replace(".png", "")
            new_img_name = f"{set_name}_{base_name}.png"
            new_lbl_name = f"{set_name}_{base_name}.txt"

            img_src = os.path.join(img_in_dir, img_file)
            lbl_src = os.path.join(lbl_in_dir, f"{base_name}.txt")

            img_dst = os.path.join(img_out_dir, new_img_name)
            lbl_dst = os.path.join(lbl_out_dir, new_lbl_name)

            shutil.copy(img_src, img_dst)
            if os.path.exists(lbl_src):
                shutil.copy(lbl_src, lbl_dst)
            else:
                open(lbl_dst, "w").close()  # empty .txt for background-only tiles

            # Add to manifest
            manifest_path = f"images/{split}/{new_img_name}"
            if split == "train":
                train_list.append(manifest_path)
            else:
                val_list.append(manifest_path)

# --- Save manifests ---
manifest_dir = os.path.join(dataset_dir, "manifests")
os.makedirs(manifest_dir, exist_ok=True)

with open(os.path.join(manifest_dir, "train_files.txt"), "w") as f:
    for item in sorted(train_list):
        f.write(f"{item}\n")

with open(os.path.join(manifest_dir, "val_files.txt"), "w") as f:
    for item in sorted(val_list):
        f.write(f"{item}\n")

print(f"✅ Dataset complete. Merged into: {dataset_dir}")
print(f"📝 Manifests saved to: {manifest_dir}")

✅ Training Sets (6): ['Bear_Creek_20250112', 'Wildcat_Creek', 'Bear_Lane', 'SFLBC', 'Flight_2', 'Flight_2_25pct']
🧪 Held-Out Test Set: Sugar_Refugia_20241112


Merging training sets: 100%|████████████████████████████| 6/6 [03:14<00:00, 32.36s/it]

✅ Dataset complete. Merged into: C:/QGIS\dataset
📝 Manifests saved to: C:/QGIS\dataset\manifests


In [6]:
# -----------------------------------------------------------
# 🏋️ YOLOv8 Training on Merged Multi-Map Dataset
#
# This step trains a segmentation model using:
# - 9 combined map sets for training and validation
# - 2 held-out sets reserved for final evaluation
#
# Training details:
# - Model type: yolov8n-seg.pt (upgradable to yolov8m-seg or yolov8l-seg)
# - Input size: 640×640
# - Dataset structure:
#     - images/train/
#     - images/val/
#     - labels/train/
#     - labels/val/
# - Data.yaml automatically generated
#     - Lists number of classes
#     - Associates image directories
# - Batch size: dynamic based on GPU capacity
# - Workers: set for safe Windows operation (default 0, can raise manually)
# - AMP (Automatic Mixed Precision): enabled for faster training with no accuracy loss
#
# Notes:
# - Background-only tiles included (empty .txt labels)
# - Model trained using the Ultraytics YOLOv8 library
# - Data manifests (train_files.txt and val_files.txt) available if needed
# - Training can be resumed or scaled easily by adjusting parameters
#
# Result:
# - Saved model weights (best.pt, last.pt) under /runs/segment/train/
# - Full training logs and metrics available for performance review
# -----------------------------------------------------------

In [ ]:
import os
from ultralytics import YOLO
import torch
torch.cuda.empty_cache()

# --- Configuration ---
dataset_dir = "C:/QGIS/dataset"
model_type = "yolov8n-seg.pt"  # or yolov8m-seg.pt
img_size = 640
epochs = 50
workers = 3
save_dir = "C:/QGIS/runs/segment/train"

# --- Class names (for 3-class segmentation) ---
class_names = ["Water", "Road", "PVeg"]

# --- Write data.yaml ---
data_yaml = os.path.join(dataset_dir, "data.yaml")
with open(data_yaml, "w") as f:
    f.write(f"path: {dataset_dir}\n")
    f.write("train: images/train\n")
    f.write("val: images/val\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

print(f"✅ data.yaml written to: {data_yaml}")

# --- Load model and train ---
model = YOLO(model_type)

model.train(
    data=data_yaml,
    imgsz=img_size,
    epochs=epochs,
    batch=-1,
    workers=workers,
    amp=True,
    device="cuda",
    save=True,
    save_period=-1,         # Save only best + last
    project=save_dir,
    name="main_model",
    verbose=True,
    plots=False,
    cache=True
)

✅ data.yaml written to: C:/QGIS/dataset\data.yaml
New https://pypi.org/project/ultralytics/8.3.124 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.111  Python-3.10.16 torch-1.12.1+cu113 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
WARNING  Upgrade to torch>=2.0.0 for deterministic training.
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/QGIS/dataset\data.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=True, device=cuda, workers=3, project=C:/QGIS/runs/segment/train, name=main_model4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_str

train: Scanning C:\QGIS\dataset\labels\train.cache... 4893 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4893/4893


WARNING  cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (5.6GB RAM): 100%|██████████| 4893/4893 [00:29<00:00, 163.36it/s]


AutoBatch: Computing optimal batch size for imgsz=640 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU) 8.00G total, 0.20G reserved, 0.02G allocated, 7.78G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output
     3264201       12.11         0.516           779           nan        (1, 3, 640, 640)                    list
     3264201       24.22         0.981         183.1           nan        (2, 3, 640, 640)                    list
     3264201       48.45         1.627         181.6           nan        (4, 3, 640, 640)                    list
     3264201       96.89         3.104         192.4           nan        (8, 3, 640, 640)                    list
     3264201       193.8         5.927         177.1           nan       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 13 for CUDA:0 5.09G/8.00G (64%) 
train: Fast image access  (ping: 0.20.0 ms, read:

...

       4/50      2.67G      1.019      2.499      1.119       1.32         59        640: 100%|██████████| 377/377 [04:
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all       2100      17112      0.572      0.329      0.313      0.204      0.569      0.307      0.278      0.145

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       5/50      2.67G     0.9664      2.383      1.034      1.282         43        640: 100%|██████████| 377/377 [04:
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP


                   all       2100      17112      0.597       0.36      0.361      0.248      0.592      0.341      0.335      0.199

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


       6/50      2.67G     0.9223      2.269     0.9883       1.26        111        640:  44%|████▍     | 165/377 [01:

In [ ]:
# Model Report

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages

# --- Paths ---
results_path = "C:/QGIS/runs/segment/train12/results.csv"
report_path = "C:/QGIS/diagnostics/model_report.pdf"
os.makedirs(os.path.dirname(report_path), exist_ok=True)

# --- Load results and clean columns ---
df = pd.read_csv(results_path)
df.columns = df.columns.str.strip()

# --- Key epoch metrics ---
final_epoch = df.iloc[-1]
best_map_epoch = df["metrics/mAP50(B)"].idxmax()
best_row = df.iloc[best_map_epoch]

# --- Basic training config (update if needed) ---
training_config = {
    "Model": "YOLOv8n-seg",
    "Input Size": "640x640",
    "Epochs": len(df),
    "Batch Size": 8,
    "Optimizer": "SGD (default)",
    "Confidence Threshold": 0.1,
    "Tile Size": "1024x1024",
    "Mask Source": "Polygon label → Raster mask via cv2.fillPoly"
}

# --- Metrics to plot ---
metrics = {
    "metrics/mAP50(B)": "mAP@0.5",
    "metrics/mAP50-95(B)": "mAP@0.5–0.95",
    "metrics/precision(B)": "Precision",
    "metrics/recall(B)": "Recall"
}
colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

# --- Create PDF report ---
with PdfPages(report_path) as pdf:
    # Title + Summary page
    plt.figure(figsize=(11, 8.5))
    plt.text(0.5, 0.78, "Model Performance Report", ha="center", fontsize=24)
    plt.text(0.5, 0.70, "Project: River and Road Segmentation", ha="center", fontsize=14)
    plt.text(0.5, 0.63, f"Best mAP@0.5: {best_row['metrics/mAP50(B)']:.3f} at epoch {best_map_epoch}", ha="center", fontsize=12)
    plt.text(0.5, 0.57, f"Final Epoch Precision: {final_epoch['metrics/precision(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.51, f"Final Epoch Recall: {final_epoch['metrics/recall(B)']:.3f}", ha="center", fontsize=12)
    plt.text(0.5, 0.45, "Report generated from YOLOv8 results.csv", ha="center", fontsize=10)
    plt.axis("off")
    pdf.savefig()
    plt.close()

    # Training configuration page
    fig, ax = plt.subplots(figsize=(11, 4))
    ax.axis("off")
    table_data = list(training_config.items())
    table = ax.table(cellText=table_data, colLabels=["Parameter", "Value"], loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(11)
    table.scale(1.2, 1.4)
    plt.title("Training Configuration", fontsize=14)
    pdf.savefig()
    plt.close()

    # Metric plots
    for i, (key, label) in enumerate(metrics.items()):
        if key in df.columns:
            plt.figure(figsize=(10, 4))
            plt.plot(df[key], marker='o', linewidth=2, color=colors[i])
            plt.title(f"{label} Over Epochs")
            plt.xlabel("Epoch")
            plt.ylabel(label)
            plt.grid(True)
            plt.tight_layout()
            pdf.savefig()
            plt.close()

    # Performance summary table
    summary_data = {
        "Metric": ["mAP@0.5", "mAP@0.5–0.95", "Precision", "Recall"],
        "Final Epoch": [
            f"{final_epoch.get('metrics/mAP50(B)', 0):.3f}",
            f"{final_epoch.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{final_epoch.get('metrics/precision(B)', 0):.3f}",
            f"{final_epoch.get('metrics/recall(B)', 0):.3f}"
        ],
        f"Best Epoch ({best_map_epoch})": [
            f"{best_row.get('metrics/mAP50(B)', 0):.3f}",
            f"{best_row.get('metrics/mAP50-95(B)', 0):.3f}",
            f"{best_row.get('metrics/precision(B)', 0):.3f}",
            f"{best_row.get('metrics/recall(B)', 0):.3f}"
        ]
    }
    summary_df = pd.DataFrame(summary_data)
    fig, ax = plt.subplots(figsize=(10, 2.5))
    ax.axis("off")
    table = ax.table(cellText=summary_df.values, colLabels=summary_df.columns, loc="center")
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    plt.title("Summary of Model Performance", fontsize=14)
    pdf.savefig()
    plt.close()

print(f"✅ PDF report saved to: {report_path}")

✅ PDF report saved to: C:/QGIS/diagnostics/model_report.pdf


In [ ]:
# -----------------------------------------------------------
# 🤖 Inference: Run YOLOv8 Segmentation on Tiled Map Images
#
# This step uses a trained YOLOv8 segmentation model to:
# - Predict object masks for each 1024×1024 tile image
# - Save polygon .txt files (YOLO format)
# - Optionally save visualized overlays
#
# It processes all tiles from the specified map folder:
# - /[Map Folder]/tiled/images/
# and outputs:
# - Polygon .txt labels to: /[Map Folder]/predictions/predict_txt/labels/
# - Optional .jpg overlays to: /[Map Folder]/predictions/predict_txt/images/
#
# Classes:
#     • 0 = Water
#     • 1 = Road
#     • 2 = PVeg (Perennial Vegetation)
#
# Parameters:
# - imgsz: Image size used during training (e.g. 640)
# - conf: Confidence threshold (e.g. 0.1 for permissive detection)
# - retina_masks: True for high-quality mask rendering
# - save_txt: Enables saving polygon .txt files (for reconstruction)
#
# Output:
# - One .txt polygon label file per tile (normalized coordinates)
# - Used in downstream mask and shapefile generation
#
# Notes:
# - This step does not create mask images directly, but polygon .txt labels
# - Masks are generated in the following step by rasterizing these polygons
# - Use consistent `imgsz` and class mapping with training
# -----------------------------------------------------------


In [1]:
from ultralytics import YOLO
import os
import torch

# --- Config ---
map_folder = "Sugar_Refugia_20241112"
base_dir = "C:/QGIS"
image_dir = os.path.join(base_dir, map_folder, "tiled", "images")
torch.cuda.empty_cache()

# Output path and label subfolder
output_name = "predict_txt"
output_dir = os.path.join(base_dir, map_folder, "predictions")

# --- Load trained YOLOv8 segmentation model ---
model_path = "C:/QGIS/runs/segment/train/main_model4/weights/best.pt"
model = YOLO(model_path)

# --- Run inference on all tiles ---
print(f"🚀 Starting inference on: {image_dir}")
model.predict(
    source=image_dir,
    imgsz=640,
    conf=0.1,
    save=False,
    save_txt=True,
    save_conf=False,
    retina_masks=True,        # ✅ Better quality masks for polygon generation
    name=output_name,
    project=output_dir,
    device="cpu",            # or "cpu"
    verbose=True              # Show internal YOLO progress
)

print(f"✅ Inference complete.")
print(f"📁 Outputs saved to: {os.path.join(output_dir, output_name)}")
print(f"📄 Label files in: {os.path.join(output_dir, output_name, 'labels')}")

🚀 Starting inference on: C:/QGIS\Sugar_Refugia_20241112\tiled\images

image 1/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_0.png: 640x640 1 Water, 1 PVeg, 886.6ms
image 2/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_1.png: 640x640 3 Waters, 5 PVegs, 1000.1ms
image 3/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_10.png: 640x640 1 Water, 951.6ms
image 4/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_100.png: 640x640 1 Water, 4 PVegs, 926.8ms
image 5/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_101.png: 640x640 1 Water, 937.3ms
image 6/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_102.png: 640x640 1 Water, 4 PVegs, 960.5ms
image 7/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_103.png: 640x640 1 Water, 1 PVeg, 788.3ms
image 8/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_104.png: 640x640 1 Water, 1 PVeg, 645.6ms
image 9/775 C:\QGIS\Sugar_Refugia_20241112\tiled\images\chip_105.png: 640x640 1 Water, 5 PVegs, 862.7ms
image 10/775 C:\QG

In [ ]:
# -----------------------------------------------------------
# 🗺️ Final Map Output: Generate Georeferenced Shapefile from Predicted Masks
#
# This step completes the pipeline by:
# - Stitching predicted mask tiles together
# - Extracting polygon features for each class:
#     • 0 = Water
#     • 1 = Road
#     • 2 = PVeg (Perennial Vegetation)
# - Converting tile-relative pixel coordinates to global raster space
# - Reprojecting to geographic coordinates using the original orthomosaic's CRS and transform
# - Writing:
#     • A raw shapefile with all per-tile polygons
#     • A post-processed, class-dissolved shapefile with clean, merged geometries
#
# Output:
# - /[Map Folder]/[Map Name] Segmentation.shp     ← raw, unmerged polygons
# - /[Map Folder]/[Map Name] Merged Segmentation.shp ← deduplicated, dissolved by class
#
# Notes:
# - Tile overlap may create redundant or fragmented predictions — the merged shapefile resolves this.
# - Class counts are printed for verification
# - All outputs are suitable for use in GIS tools like QGIS or ArcGIS
# -----------------------------------------------------------


In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import geopandas as gpd
from shapely.geometry import Polygon
import rasterio
from tqdm import tqdm

# --- Config ---
map_folder = "Sugar_Refugia_20241112"
base_dir = f"C:/QGIS/{map_folder}"
mask_dir = os.path.join(base_dir, "predictions", "generated_masks")
tile_metadata_path = os.path.join(base_dir, "tiled", "tile_metadata.csv")
raster_shape_path = os.path.join(base_dir, "tiled", "raster_shape.txt")
tif_path = os.path.join(base_dir, f"{map_folder}.tiff")
raw_shapefile = os.path.join(base_dir, f"{map_folder} Segmentation.shp")
clean_shapefile = os.path.join(base_dir, f"{map_folder} Merged Segmentation.shp")
chip_size = 1024

# --- Load metadata and georeferencing ---
tile_meta = pd.read_csv(tile_metadata_path)
with open(raster_shape_path, "r") as f:
    height, width = map(int, f.read().strip().split(","))

with rasterio.open(tif_path) as src:
    transform = src.transform
    crs = src.crs

# --- Extract polygons from mask tiles ---
features = []
class_counts = {0: 0, 1: 0, 2: 0}

for _, row in tqdm(tile_meta.iterrows(), total=len(tile_meta), desc="Stitching masks"):
    fname, x, y = row["filename"], int(row["x"]), int(row["y"])
    mask_path = os.path.join(mask_dir, fname)
    if not os.path.exists(mask_path):
        continue

    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if mask is None:
        continue

    for class_id in np.unique(mask):
        if class_id == 0 and np.all(mask == 0):
            continue

        binary = (mask == class_id).astype(np.uint8)
        kernel = np.ones((3, 3), np.uint8)
        binary = cv2.dilate(binary, kernel, iterations=1)

        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

        for cnt in contours:
            if len(cnt) < 3:
                continue

            pts = cnt.reshape(-1, 2)
            pts[:, 0] += x
            pts[:, 1] += y

            geo_pts = [rasterio.transform.xy(transform, y_, x_, offset='center') for x_, y_ in pts]
            poly = Polygon(geo_pts)
            if not poly.is_valid or poly.area == 0:
                continue

            features.append({"geometry": poly, "class_id": int(class_id)})
            class_counts[class_id] += 1

# --- Save raw shapefile ---
if not features:
    print("⚠️ No valid polygons found. Skipping shapefile export.")
else:
    gdf = gpd.GeoDataFrame(features, crs=crs)
    gdf.to_file(raw_shapefile)
    print(f"✅ Raw shapefile saved to: {raw_shapefile}")
    print(f"📊 Raw class counts: Water={class_counts[0]}, Road={class_counts[1]}, PVeg={class_counts[2]}")

    # --- Post-process: dissolve and simplify ---
    print("🧼 Post-processing: dissolving by class_id and simplifying...")
    dissolved = gdf.dissolve(by="class_id", as_index=False)

    # Optional: simplify to remove tiny jagged edges
    dissolved["geometry"] = dissolved["geometry"].simplify(tolerance=0.2, preserve_topology=True)

    # Save cleaned version
    dissolved.to_file(clean_shapefile)
    print(f"✅ Merged shapefile saved to: {clean_shapefile}")


Stitching masks:  66%|█████████████████████████████████████████▌                     | 512/775 [10:45<04:54,  1.12s/it]

In [ ]:
# -----------------------------------------------------------
# 🖼️ Final Preview: Create Overlay Image of Shapefile on Orthomosaic
#
# This optional step generates a lightweight PNG preview by:
# - Downscaling the orthomosaic TIFF to create a visual background
# - Loading the final merged segmentation shapefile
# - Transforming polygon geometries from geographic to pixel coordinates
# - Overlaying colored polygons for each class onto the image:
#     • 0 = Water (Blue)
#     • 1 = Road (Yellow)
#     • 2 = PVeg (Green)
#
# Output:
# - /[Map Folder]/preview_overlay_from_shapefile.png ← downscaled visual summary
#
# Notes:
# - Output is suitable for diagnostics or client preview
# - Image scale is adjustable via the `scale` variable (default = 10%)
# - Preserves spatial accuracy and class separation with color-coded RGBA masks
# -----------------------------------------------------------


In [4]:
import os
import rasterio
import numpy as np
import geopandas as gpd
import cv2
from PIL import Image
from shapely.geometry import Polygon
from shapely.affinity import affine_transform

# --- Config ---
map_folder = "Sugar_Refugia_20241112"
base_dir = f"C:/QGIS/{map_folder}"

tif_path = os.path.join(base_dir, f"{map_folder}.tiff")
shapefile_path = os.path.join(base_dir, f"{map_folder} Merged Segmentation.shp")
output_path = os.path.join(base_dir, "preview_overlay_from_shapefile.png")
scale = 0.10  # 10% preview resolution

# --- Color map (RGBA) ---
COLOR_MAP = {
    0: (0, 128, 255, 128),     # Water → blue
    1: (255, 255, 0, 128),     # Road → yellow
    2: (0, 255, 0, 128),       # PVeg → green
}

# --- Load base orthomosaic and downscale ---
with rasterio.open(tif_path) as src:
    height, width = src.height, src.width
    preview_height = int(height * scale)
    preview_width = int(width * scale)

    image = src.read(
        indexes=[1, 2, 3],
        out_shape=(3, preview_height, preview_width),
        resampling=rasterio.enums.Resampling.bilinear
    )
    image = np.transpose(image, (1, 2, 0))  # CHW → HWC
    image = np.clip(image, 0, 255).astype(np.uint8)

    transform = src.transform
    inv_transform = [
        1 / transform.a, 0,
        0, 1 / transform.e,
        -transform.c / transform.a,
        -transform.f / transform.e
    ]

# --- Create RGBA overlay ---
overlay = np.zeros((preview_height, preview_width, 4), dtype=np.uint8)

# --- Load shapefile and rasterize polygons into preview image space ---
gdf = gpd.read_file(shapefile_path)

for _, row in gdf.iterrows():
    geom = row.geometry
    class_id = int(row["class_id"])
    rgba = COLOR_MAP.get(class_id)
    if rgba is None:
        continue

    # Transform geographic coordinates to pixel coords (preview scale)
    preview_poly = affine_transform(geom, inv_transform)
    scaled_poly = affine_transform(preview_poly, [scale, 0, 0, scale, 0, 0])

    if not isinstance(scaled_poly, Polygon):
        continue

    exterior = np.array(scaled_poly.exterior.coords).astype(np.int32)
    cv2.fillPoly(overlay, [exterior], color=rgba)

# --- Convert and blend ---
base_pil = Image.fromarray(image).convert("RGBA")
overlay_pil = Image.fromarray(overlay, mode="RGBA")
composite = Image.alpha_composite(base_pil, overlay_pil)

composite.save(output_path)
print(f"✅ Preview image saved to: {output_path}")

✅ Preview image saved to: C:/QGIS/Sugar_Refugia_20241112\preview_overlay_from_shapefile.png
